In [ ]:
from datasets import load_dataset

ds = load_dataset("NebulaByte/E-Commerce_Customer_Support_Conversations")

In [ ]:
ds

In [ ]:
df = ds["train"].to_pandas()
df.head()

In [ ]:
def build_document(row):
    return f"""
Issue Area: {row['issue_area']}
Issue Category: {row['issue_category']}
Issue Subcategory: {row['issue_sub_category']}

Product: {row['product_category']} -> {row['product_sub_category']}
Customer Sentiment: {row['customer_sentiment']}
Issue Complexity: {row['issue_complexity']}

Conversation:
{row['conversation']}

Resolution Summary:
This issue was resolved by following the agent guidance in the conversation above.
""".strip()


In [ ]:
documents = df.apply(build_document, axis=1).tolist()
documents

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

chunks = []
for doc in documents:
    chunks.extend(splitter.split_text(doc))


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

model = SentenceTransformer("all-MiniLM-L6-v2")

chunk_embeddings = model.encode(chunks, show_progress_bar=True)

index = faiss.IndexFlatL2(chunk_embeddings.shape[1])
index.add(np.array(chunk_embeddings))


In [ ]:
def retrieve_chunks(query, k=4):
    q_emb = model.encode([query])
    _, idxs = index.search(np.array(q_emb), k)
    return [chunks[i] for i in idxs[0]]


In [ ]:
def build_prompt(context_chunks, user_query):
    context = "\n\n".join(context_chunks)

    return f"""
You are a customer support agent.

Use ONLY the information below to answer the user.
If the answer is not present, say you are unsure.
If related information is present, use it to help the user.

Support Documentation from past customer interactions:
{context}

User Question:
{user_query}

Answer clearly and step-by-step.
"""


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
groqAPIKey = os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_groq import ChatGroq
chatModel = ChatGroq(model = 'meta-llama/llama-4-scout-17b-16e-instruct',api_key=groqAPIKey)

In [ ]:
user_query = "How can I reset my password if I forgot it?"

In [ ]:
context = retrieve_chunks(user_query)
response = chatModel.invoke(build_prompt(context, user_query))


In [ ]:
response.content

In [ ]:
user_query = "Whats the last question I asked?"
context = retrieve_chunks(user_query)
response = chatModel.invoke(build_prompt(context, user_query))
response.content